# TensorTools Runtime Demo

Demonstrates the usage of the runtime using a simple autoencoder model.

In [ ]:
# Force matplotlib to use inline rendering
%matplotlib inline

import os
import sys

# add path to libraries for ipython
sys.path.append(os.path.expanduser("~/libs"))

import numpy as np
import tensorflow as tf
import tensortools as tt

In [ ]:
BATCH_SIZE = 16
REG_LAMBDA = 5e-4
INITIAL_LR = 0.005
LR_DECAY_STEP_INTERVAL = 10000
LR_DECAY_FACTOR = 0.5
NUM_GPUS = 2

In [ ]:
dataset_train = tt.datasets.moving_mnist.MovingMNISTTrainDataset(input_shape=[1, 64, 64, 1],
                                                                 target_shape=[1, 64, 64, 1])
dataset_valid = tt.datasets.moving_mnist.MovingMNISTValidDataset(input_shape=[1, 64, 64, 1],
                                                                 target_shape=[1, 64, 64, 1])
#dataset_test = tt.datasets.moving_mnist.MovingMNISTTestDataset(input_seq_length=1, target_seq_length=0)

In [ ]:
dataset_train = tt.datasets.mnist.MNISTTrainDataset()
dataset_valid = tt.datasets.mnist.MNISTValidDataset()
#dataset_test = tt.datasets.mnist.MNISTTestDataset()

In [ ]:
dataset_train = tt.datasets.ucf11.UCF11TrainDataset(input_seq_length=1, target_seq_length=1,
                                                    image_size=(60, 80, 1),
                                                    num_threads=8,
                                                    do_distortion=False)
#dataset_valid = tt.datasets.ucf11.UCF11TrainDataset(input_seq_length=1, target_seq_length=1,
#                                                    image_size=(60, 80, 1),
#                                                    num_threads=8,
#                                                    do_distortion=False)
dataset_valid = None

In [ ]:
class SimpleAutoencoderModel(tt.model.AbstractModel):    
    def __init__(self, reg_lambda=0.0):
        super(SimpleAutoencoderModel, self).__init__(reg_lambda)
        
    @tt.utils.attr.override
    def inference(self, inputs, targets, is_training=True,
                  device_scope=None, memory_device=None):
        input_shape = inputs.get_shape()
        x = tf.contrib.layers.flatten(inputs)
        encoded = tt.network.fc("FC_Enc", x, 64,
                                weight_init=tf.contrib.layers.xavier_initializer(),
                                bias_init=0.0,
                                activation=tf.nn.relu,
                                device=memory_device)
        representation = encoded
        decoded = tt.network.fc("FC_Dec", representation, x.get_shape()[1],
                                weight_init=tf.contrib.layers.xavier_initializer(),
                                bias_init=0.0,
                                activation=tf.nn.sigmoid,
                                device=memory_device)

        return tf.reshape(decoded, [-1] + [int(x) for x in input_shape[1:]])
    
    @tt.utils.attr.override
    def loss(self, predictions, targets):
        return tt.loss.mse(predictions, targets)

In [ ]:
runtime = tt.core.DefaultRuntime()
#runtime = tt.core.MultiGpuRuntime(NUM_GPUS)
runtime.register_datasets(dataset_train, dataset_valid)
runtime.register_model(SimpleAutoencoderModel(reg_lambda=REG_LAMBDA))
runtime.build(INITIAL_LR,
              LR_DECAY_STEP_INTERVAL,
              LR_DECAY_FACTOR,
              is_autoencoder=True)

In [ ]:
runtime.train(BATCH_SIZE, steps=1000)

In [ ]:
runtime.validate(batch_size=50)

In [ ]:
runtime.test(batch_size=50)

In [ ]:
image_shape = dataset_train.input_shape
if len(image_shape) == 4: # MovingMNIST / UCF11
    fake_input_batch = np.random.rand(6,1,image_shape[-3],image_shape[-2],image_shape[-1])
else: # MNIST
    fake_input_batch = np.random.rand(6,image_shape[-3],image_shape[-2],image_shape[-1])
predictions = runtime.predict(fake_input_batch)

print(predictions.shape)

print(fake_input_batch.min(), fake_input_batch.max())
print(predictions.min(), predictions.max())

tt.visualization.display_batch((fake_input_batch + 1) * 127.5, title="Inputs")
tt.visualization.display_batch(predictions * 255, title="Predictions")

In [ ]:
runtime.close()